<a href="https://colab.research.google.com/github/SaidaKarimova9/DSD-Final-Project/blob/main/Final_version_of_the_second_model_CorAl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy pandas matplotlib torch torchvision


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import zipfile
import os

source_zip = '/content/drive/MyDrive/ML-2024/AFE.zip'
target_zip = '/content/drive/MyDrive/ML-2024/ExpW.zip'



In [ ]:
# Unzip datasets
with zipfile.ZipFile(source_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/AFE')

with zipfile.ZipFile(target_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/ExpW')

In [ ]:
#Load and Preprocess Data

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

source_data_dir = '/content/AFE/AFE'
target_data_dir = '/content/ExpW/ExpW'

image_datasets = {
    'source_train': ImageFolder(os.path.join(source_data_dir, 'train'), data_transforms['train']),
    'source_test': ImageFolder(os.path.join(source_data_dir, 'test'), data_transforms['test']),
    'target_train': ImageFolder(os.path.join(target_data_dir, 'train'), data_transforms['train']),
    'target_test': ImageFolder(os.path.join(target_data_dir, 'test'), data_transforms['test'])
}

dataloaders = {
    'source_train': DataLoader(image_datasets['source_train'], batch_size=32, shuffle=True, num_workers=4),
    'source_test': DataLoader(image_datasets['source_test'], batch_size=32, shuffle=False, num_workers=4),
    'target_train': DataLoader(image_datasets['target_train'], batch_size=32, shuffle=True, num_workers=4),
    'target_test': DataLoader(image_datasets['target_test'], batch_size=32, shuffle=False, num_workers=4)
}


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

source_data_dir = '/content/AFE/AFE'
target_data_dir = '/content/ExpW/ExpW'

image_datasets = {
    'source_train': ImageFolder(os.path.join(source_data_dir, 'train'), data_transforms['train']),
    'source_test': ImageFolder(os.path.join(source_data_dir, 'test'), data_transforms['test']),
    'target_train': ImageFolder(os.path.join(target_data_dir, 'train'), data_transforms['train']),
    'target_test': ImageFolder(os.path.join(target_data_dir, 'test'), data_transforms['test'])
}

dataloaders = {
    'source_train': DataLoader(image_datasets['source_train'], batch_size=32, shuffle=True, num_workers=4),
    'source_test': DataLoader(image_datasets['source_test'], batch_size=32, shuffle=False, num_workers=4),
    'target_train': DataLoader(image_datasets['target_train'], batch_size=32, shuffle=True, num_workers=4),
    'target_test': DataLoader(image_datasets['target_test'], batch_size=32, shuffle=False, num_workers=4)
}

# Define model
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(image_datasets['source_train'].classes))

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Define CORAL loss function
def coral_loss(source, target):
    d = source.size(1)
    source_cov = (source.t() @ source) / (source.size(0) - 1)
    target_cov = (target.t() @ target) / (target.size(0) - 1)

    loss = torch.sum((source_cov - target_cov) ** 2) / (4 * d ** 2)
    return loss

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 214MB/s]


In [ ]:
# Train the model with CORAL
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    # Train on source domain
    for inputs, labels in dataloaders['source_train']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        classification_loss = criterion(outputs, labels)
#
        target_inputs, _ = next(iter(dataloaders['target_train']))
        target_inputs = target_inputs.to(device)

        target_outputs = model(target_inputs)
        coral_loss_value = coral_loss(outputs, target_outputs)

        loss = classification_loss + coral_loss_value
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloaders['source_train'].dataset)
    epoch_acc = running_corrects.double() / len(dataloaders['source_train'].dataset)

    print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

Epoch 0/9, Loss: 1.0598, Acc: 0.6381
Epoch 1/9, Loss: 0.7291, Acc: 0.7610
Epoch 2/9, Loss: 0.6518, Acc: 0.7880
Epoch 3/9, Loss: 0.5965, Acc: 0.8121
Epoch 4/9, Loss: 0.5556, Acc: 0.8236
Epoch 5/9, Loss: 0.5369, Acc: 0.8297
Epoch 6/9, Loss: 0.5139, Acc: 0.8387
Epoch 7/9, Loss: 0.4969, Acc: 0.8452
Epoch 8/9, Loss: 0.4860, Acc: 0.8484
Epoch 9/9, Loss: 0.4732, Acc: 0.8508


In [ ]:
model_save_path = '/content/drive/MyDrive/coral_model.pth'
torch.save(model.state_dict(), model_save_path)


In [ ]:
model.load_state_dict(torch.load(model_save_path))
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
source_corrects = 0
target_corrects = 0

# Evaluate on source test data
with torch.no_grad():
    for inputs, labels in dataloaders['source_test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        source_corrects += torch.sum(preds == labels.data)

source_acc = source_corrects.double() / len(dataloaders['source_test'].dataset)

# Evaluate on target test data
with torch.no_grad():
    for inputs, labels in dataloaders['target_test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        target_corrects += torch.sum(preds == labels.data)

target_acc = target_corrects.double() / len(dataloaders['target_test'].dataset)

print(f'Source Test Accuracy: {source_acc:.4f}')
print(f'Target Test Accuracy: {target_acc:.4f}')


Source Test Accuracy: 0.9137
Target Test Accuracy: 0.5565


In [ ]:
#
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(image_datasets['target_train'].classes))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    # Train on target domain
    for inputs, labels in dataloaders['target_train']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloaders['target_train'].dataset)
    epoch_acc = running_corrects.double() / len(dataloaders['target_train'].dataset)

    print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

# Save the model
oracle_model_save_path = '/content/drive/MyDrive/oracle_model.pth'
torch.save(model.state_dict(), oracle_model_save_path)


Epoch 0/9, Loss: 1.2607, Acc: 0.5367
Epoch 1/9, Loss: 1.1051, Acc: 0.6023
Epoch 2/9, Loss: 1.0613, Acc: 0.6213
Epoch 3/9, Loss: 1.0240, Acc: 0.6351
Epoch 4/9, Loss: 1.0028, Acc: 0.6432
Epoch 5/9, Loss: 0.9832, Acc: 0.6489
Epoch 6/9, Loss: 0.9722, Acc: 0.6538
Epoch 7/9, Loss: 0.9478, Acc: 0.6629
Epoch 8/9, Loss: 0.9396, Acc: 0.6630
Epoch 9/9, Loss: 0.9241, Acc: 0.6699


In [ ]:
# Load the trained model
model.load_state_dict(torch.load(oracle_model_save_path))
model.eval()

target_corrects = 0

# Evaluate on target test data
with torch.no_grad():
    for inputs, labels in dataloaders['target_test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        target_corrects += torch.sum(preds == labels.data)

target_acc = target_corrects.double() / len(dataloaders['target_test'].dataset)
print(f'Oracle Target Test Accuracy: {target_acc:.4f}')
print(f'Oracle Target Test Accuracy: {target_acc * 100:.2f}%')


Oracle Target Test Accuracy: 0.7724
Oracle Target Test Accuracy: 77.24%
